## Copied from original data download notebook.

The following lines of code are copied straight from the original data download notebook with only a few modifications so that they utilize the new 2018 python scripts. If you made any modifications to the original notebook in order to get it to run, you will need to repeat those steps. Otherwise, run everything as is.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from src.data import data_collection_2018

In [5]:
data_collection_2018.download_data_and_load_into_sql()

Successfully downloaded ZIP file
    https://www2.census.gov/programs-surveys/acs/data/pums/2014/5-Year/csv_pwa.zip
    
Successfully downloaded ZIP file
    https://www2.census.gov/programs-surveys/acs/data/pums/2015/5-Year/csv_pwa.zip
    
Successfully downloaded ZIP file
    https://www2.census.gov/programs-surveys/acs/data/pums/2016/5-Year/csv_pwa.zip
    
Successfully downloaded ZIP file
    https://www2.census.gov/programs-surveys/acs/data/pums/2018/5-Year/csv_pwa.zip
    


BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY pums_2014, line 1: "P,2010000000034,01,01802,-0009,53,1094136,00014,46,1,,,6,2,2,2,2,2,,,2,,2,2,,,1,2,2,2,2,2,2,000000,,..."


In [18]:
import pandas as pd
import psycopg2

DBNAME = "opportunity_youth"

#Retrieving postgres info for MEF
from src.data import local
PORT = local.port
USER = local.user
HOST = local.host
PASSWORD = local.password
conn = psycopg2.connect(dbname=DBNAME, user=USER, password=PASSWORD, port=PORT, host=HOST)


# conn = psycopg2.connect(dbname=DBNAME)

### OY youth estimate for 2017

In [21]:
query = """
SELECT agep, sch, esr, pwgtp
FROM pums_2017
WHERE puma in
    (SELECT pn.puma
    FROM puma_names_2010 AS pn
    WHERE pn.puma_name LIKE '%King%South%'
    AND pn.puma_name NOT LIKE '%Northwest%')
AND agep BETWEEN 16 and 24
AND esr in ('3', '6')
AND sch = '1'
"""
skc_search = pd.read_sql(query, conn)
skc_search.pwgtp.sum()

6723.0

### OY youth estimate for 2018

In [20]:
query = """
SELECT agep, sch, esr, pwgtp
FROM pums_2018
WHERE puma in
    (SELECT pn.puma
    FROM puma_names_2010 AS pn
    WHERE pn.puma_name LIKE '%King%South%'
    AND pn.puma_name NOT LIKE '%Northwest%')
AND agep BETWEEN 16 and 24
AND esr in ('3', '6')
AND sch = '1'
"""
skc_search = pd.read_sql(query, conn)
skc_search.pwgtp.sum()

6642.0

In [ ]:
conn.close()